In [ ]:
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant

In [ ]:
from google.colab import drive
import os
from pathlib import Path
import pandas as pd
import re
import numpy as np
import enchant
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import BlanklineTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
imdb = pd.read_csv(gdrive/'imdb1m-posts.txt', sep = '\t', names = ["postId", "userId", "title", "text"])
imdb['text'] = imdb['text'].apply(str)
imdb = imdb.drop("postId", axis = 1)
imdb = imdb.drop("title", axis = 1)



In [ ]:
from nltk.metrics import edit_distance

class SpellingReplacer:
    def __init__(self, dict_name='en_GB', max_dist=3):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = 3

    def replace(self, word):
        if self.spell_dict.check(word):
            return 0
        suggestions = self.spell_dict.suggest(word)

        if suggestions and edit_distance(word, suggestions[0]) <= self.max_dist:
            return 1
        else:
            return 0


def spell_check(word_list):
    number_wrong = 0
    # number_wrong = np.sum([SpellingReplacer.replace(item) for item in word_list])
    
    for item in word_list:
        replacer = SpellingReplacer()
        number_wrong = number_wrong + replacer.replace(item)

    return number_wrong

In [ ]:
def get_misspell(text):
  return spell_check(text)

def add_stylometry(df):
  tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
  df["Tokens"] =  df.text.apply(nltk.word_tokenize)
  sent_tokenizer = nltk.tokenize.sent_tokenize
  df["Characters"] = df.text.str.len() 
  df["Words"] = df.text.apply(tokenizer.tokenize).apply(len)
  df["Captials"] = df.text.str.findall(r'[A-Z]').str.len()
  df["Paragraphs"] = df.text.apply(BlanklineTokenizer().tokenize).apply(len)
  df["Sentences"] = df.text.apply(sent_tokenizer).apply(len)
  df["CharactersPerWord"] = df["Characters"] / df["Words"]
  df["WordsPerSentence"] = df["Words"] / df["Sentences"]
  df["SentencesPerParagraph"] = df["Sentences"] / df["Paragraphs"]
  df["WordsinFirstParagraph"] = df.text.apply(BlanklineTokenizer().tokenize).str[0].apply(len)
  df["WordsinLastParagraph"] = df.text.apply(BlanklineTokenizer().tokenize).str[-1].apply(len)
  df["First/Last"] = df["WordsinFirstParagraph"] / df["WordsinLastParagraph"]
  df["Misspellings"] = df.Tokens.apply(get_misspell)
  df["MisspellingRatio"] = df.Misspellings / df.Words
  df = df.drop("Tokens", axis = 1)
  return df

imdb_metrics = add_stylometry(imdb)
imdb_metrics = imdb_metrics.drop("text", axis = 1)

In [ ]:
imdb_metrics


,userID,Characters,Words,Captials,Paragraphs,Sentences,CharactersPerWord,WordsPerSentence,SentencesPerParagraph,WordsinFirstParagraph,WordsinLastParagraph,First/Last
0,author,4,1,0,1,1,4.000,1.0,1.0,4,4,1.0
1,1,5099,1000,0,1,1,5.099,1000.0,1.0,5099,5099,1.0
2,1,5369,1000,0,1,1,5.369,1000.0,1.0,5369,5369,1.0
3,1,5028,1000,0,1,1,5.028,1000.0,1.0,5028,5028,1.0
4,1,5034,1000,0,1,1,5.034,1000.0,1.0,5034,5034,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
53674,50,4921,1000,0,1,1,4.921,1000.0,1.0,4921,4921,1.0
53675,50,4866,1000,0,1,1,4.866,1000.0,1.0,4866,4866,1.0
53676,50,4707,1000,0,1,1,4.707,1000.0,1.0,4707,4707,1.0
53677,50,4598,1000,0,1,1,4.598,1000.0,1.0,4598,4598,1.0


In [ ]:
imdb_metrics.to_csv(gdrive/"imdb_metrics.csv", sep='\t')
